In [1]:
"""给定一个字符串 s 和一个非空字符串 p，找到 s 中所有是 p 的字母异位词的子串，返回这些子串的起始索引。

字符串只包含小写英文字母，并且字符串 s 和 p 的长度都不超过 20100。

说明：

字母异位词指字母相同，但排列不同的字符串。
不考虑答案输出的顺序。"""

'给定一个字符串 s 和一个非空字符串 p，找到 s 中所有是 p 的字母异位词的子串，返回这些子串的起始索引。\n\n字符串只包含小写英文字母，并且字符串 s 和 p 的长度都不超过 20100。\n\n说明：\n\n字母异位词指字母相同，但排列不同的字符串。\n不考虑答案输出的顺序。'

In [2]:
"""
【思路】对s的每个元素i，先复制p到k，以i为首的len(p)个元素，逐个判断是否在k中，如果在就将这个元素从k中删除，不在就结束i为首的这次循环
如果一次遍历中，k能为空（j能判断结束），则加入ret

【优化】
1. i的循环变量可以在提前结束，不用循环到最后，只要循环到len(s)-len(p)
2. 一旦发现j不在k中，结束这次遍历，i进入下次遍历

【错误】
1. 对正在循环的数组做删除是十分危险的
2. set作为滑动窗口：如果p中有大量重复元素，p='aaaaaa',将会判断出错，如s="a"，也会认为是正确结果
3. 提前终止循环使得j可能遍历不到p的最后一个元素，这正是判断要不要加进ret的条件

【结果】
超时

【复杂度】
时间复杂度：O(N * K)
空间复杂度：O(K)
K=len(p) 如果K很大，接近N的话，那么复杂度是O(N^2)级别的，但是N最大为20100，
这种情况下的N^2的算法不能接受。
"""


class Solution:
    def findAnagrams(self, s: str, p: str):
        ret = []
        for i in range(len(s) - len(p) + 1):
            k = list(p)
            for j in range(len(p)):
                if i + j < len(s) and s[i + j] in k:
                    k.remove(s[i + j])
                else:
                    break
                if j == len(p) - 1:
                    ret.append(i)
        return ret


In [13]:
"""
【思路】
注意到字符串中仅有小写英文字母
使用hashmap作为滑动窗口而不是list。

【错误】
虽然现在滑动窗口是hashset，但是每次遍历s都新建一个hashset，总复杂度
仍然是O(N * K), 这不是真正的滑动窗口，而是每次滑动，都建了一个新窗口。

**滑动窗口要义**：不仅仅是滑动，而且每次滑动，窗口大部分内容不变来减少复杂度，仅改变少量因为滑动
而增删的元素，来更新滑动窗口。

【超时】
"""


class Solution:
    def findAnagrams(self, s: str, p: str):
        ret = []
        from collections import defaultdict
        p_d = defaultdict(int)
        for i in p:
            p_d[i] += 1
        for j in range(len(s) - len(p) + 1):
            s_d = defaultdict(int)
            for k in range(len(p)):
                s_d[s[k + j]] += 1
            try:
                assert s_d == p_d
            except AssertionError:
                continue
            ret.append(j)
        return ret


In [3]:
"""
根据上面的几次错误示范：
滑动窗口的大部分内容是不动的，只更新因为滑动而造成的元素的增删。
【思路】
1. 将p先存成hashmap p_d
2. [i,j]是s上的滑动窗口，将滑动窗口的内容存入hashmap s_d
3. 将s[j]加入滑动窗口，更新滑动窗口
4. 如果s_d==p_d，说明此时滑动窗口是满足（异位词）
5. 将滑动窗口左侧s[i]去除，以便可以添加新的s[j]
6. i++, j++

【注意】 上面思路忽略了一些边界条件。j元素实际上没有使用，使用了foreach遍历的s，但原理一样
1. 滑动窗口的更新 需要添加元素 和 删除元素。
2. 滑动窗口的边界，i,j的含义和 i，j更新的顺序 以及 i,j的初值， 对比注释代码和283 Move Zeros的边界辨析。
"""


# class Solution:
#     def findAnagrams(self, s: str, p: str):
#         ret = []
#         i, j = -len(p), 0
#         p_d = {}
#         for m in p:
#             p_d[m] = p_d.get(m, 0) + 1
#         s_d = {}
#         for n in s:
#             s_d[n] = s_d.get(n, 0) + 1
#             i += 1
#             if i >= 0:
#                 if s_d == p_d:
#                     ret.append(i)
#                 s_d[s[i]] = s_d[s[i]] - 1
#                 if s_d[s[i]] == 0:
#                     del s_d[s[i]]
# 
#         return ret

class Solution:
    def findAnagrams(self, s: str, p: str):
        ret = []
        i, j = -len(p) + 1, 0
        p_d = {}
        for m in p:
            p_d[m] = p_d.get(m, 0) + 1
        s_d = {}
        for n in s:
            s_d[n] = s_d.get(n, 0) + 1
            if i >= 0:  # s_d有len(p)的长度
                if s_d == p_d:
                    ret.append(i)
                s_d[s[i]] = s_d[s[i]] - 1
                if s_d[s[i]] == 0:
                    del s_d[s[i]]
            i += 1
        return ret


In [ ]:
"""
【核心】不仅滑动窗口是增量变化，和目标对比，也是增量对比，进一步降低复杂度，这是真正的O(N)复杂度
【思路】
仿照76的做法，s上使用[i,j)维护一个滑动窗口，p上维护一个频数字典
每当s上滑动窗口遍历到一个字母，p字典对应字母的频数减1
当p字典所有字母对应的频数<=0时，说明s上的滑动窗口包含p中所有字母
* 如果滑动窗口遍历的字母不在p中，p字典也会将相应的字母频数减1，字母频数可能为负数，负数的含义是滑动窗口中包含的无效字母出现的频数的相反数。
* 每次判断s上的滑动窗口包含p中所有字母需要遍历p字典，使用p_len辅助，那么只要O(1)的复杂度就能判断，p_len是记录p字典中频数大于1的数目，
    随着滑动窗口的遍历，频数减小，p_len减小，当p_len==0,说明滑动窗口包含p中所有字母。
但是，包含p不代表就是异构词，同时要求滑动窗口的长度和p的长度一样，这样才是异构词
* 滑动窗口中可能包含不是p中的无效字母，这是要除去的，也就是要缩小滑动窗口。
* 缩小到什么时候？缩小到某个左边界对应字母的p字典的频数为0，频数为负数的字母一定是无效字母，频数为正数说明不完全包含p的该字母，频数为0说明正好包含p中该字母
* 此时，如果滑动窗口的长度等于p的长度，它一定是解。
* 继续缩小窗口，以便形成新的滑动窗口。
【错误】
* 边界问题又错了，调用p_d[s[i]]前先i++,这样s[i]已经改变了，不应该。
"""
from collections import defaultdict


class Solution:
    def findAnagrams(self, s: str, p: str):
        ret = []
        i, j = 0, 0
        # [i,j)是异位词
        p_d = defaultdict(int)
        for k in p:
            p_d[k] += 1
        p_len = len(p)
        while j < len(s):
            if p_d[s[j]] > 0:
                p_len -= 1
            p_d[s[j]] -= 1
            if p_len == 0:  # 窗口中已经包含p
                while p_d[s[i]] < 0:
                    p_d[s[i]] += 1
                    i += 1
                # print(i, j, p_d)
                # j已经被包含，此时 [i,j]满足长度 == len(p)的话，就是异构词
                if j - i + 1 == len(p):
                    ret.append(i)

                p_d[s[i]] += 1
                p_len += 1
                i += 1
            j += 1

        return ret


In [5]:
s = Solution()
s.findAnagrams("baa", "aa")

[1]